# 4. Query Decomposition (Divide & Conquer)

**What:** Break complex question into simpler sub-questions

**Why:** Answer each part independently, then synthesize

**When:** Multi-part questions, comparisons, needs info from multiple docs

**LLM Calls:** 3-5 (decompose + sub-answers + synthesis)

**Flow:**
```
Complex Question → [Sub-Q1, Sub-Q2, ...] → Answer each → Synthesize
```

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from config import model, setup_vectorstore, get_retriever, format_docs

## Decompose Query

In [ ]:
def decompose_query(question, max_sub=4):
    template = """Break this complex question into {max_sub} simple sub-questions.
Each should ask about ONE specific aspect.

Complex: {question}

Sub-questions:"""
    
    prompt = ChatPromptTemplate.from_template(template)
    chain = prompt | model | StrOutputParser()
    response = chain.invoke({"question": question, "max_sub": max_sub})
    
    sub_questions = []
    for line in response.strip().split("\n"):
        line = line.strip()
        if line and len(line) > 5:
            if line[0].isdigit():
                line = line.split(". ", 1)[-1]
            sub_questions.append(line)
    
    return sub_questions[:max_sub]

## Answer Sub-Question

In [ ]:
def answer_sub_question(sub_q, retriever):
    docs = retriever.invoke(sub_q)
    context = format_docs(docs[:3])
    
    template = """Answer concisely based on context.

Context: {context}

Question: {question}

Answer:"""
    
    prompt = ChatPromptTemplate.from_template(template)
    chain = prompt | model | StrOutputParser()
    return chain.invoke({"context": context, "question": sub_q}), docs

## Complete Decomposition RAG

In [ ]:
def decomposition_rag(question, retriever):
    print(f"Complex: {question}\n")
    
    sub_questions = decompose_query(question)
    print("Sub-questions:")
    for i, sq in enumerate(sub_questions):
        print(f"  {i+1}. {sq}")
    
    # Answer each
    sub_answers = []
    for sq in sub_questions:
        answer, _ = answer_sub_question(sq, retriever)
        sub_answers.append({"question": sq, "answer": answer})
    
    # Synthesize
    qa_pairs = "\n".join([f"Q: {sa['question']}\nA: {sa['answer']}" for sa in sub_answers])
    
    template = """Synthesize a comprehensive answer from these Q&A pairs.

{qa_pairs}

Original: {question}

Final answer:"""
    
    prompt = ChatPromptTemplate.from_template(template)
    chain = prompt | model | StrOutputParser()
    final = chain.invoke({"qa_pairs": qa_pairs, "question": question})
    
    print(f"\nFinal Answer: {final}")
    return final

## Test

In [ ]:
vectorstore = setup_vectorstore()
retriever = get_retriever(vectorstore, k=3)

test_questions = [
    "Compare Otabek's education and work experience",
    "What is the difference between Graph DTA and Graph DF?"
]

for q in test_questions:
    print("="*60)
    decomposition_rag(q, retriever)
    print()